# Kütüphanelerin Yüklenmesi


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt


In [3]:
DATASET_DIR = "C:/Users/atunc/OneDrive/Belgeler/lung-cancer/dataset"

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,   # %80 train - %20 validation
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)


Found 15000 files belonging to 3 classes.
Using 12000 files for training.
Found 15000 files belonging to 3 classes.
Using 3000 files for validation.


In [4]:
AUTOTUNE = tf.data.AUTOTUNE

# Normalize etme (0-1 aralığına)
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Performansı artırmak için prefetch
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [5]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(224,224,3)),
    
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_ds.element_spec[1].shape[0], activation='softmax')  # sınıf sayısı
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


ValueError: Shapes used to initialize variables must be fully-defined (no `None` dimensions). Received: shape=(128, None) for variable path='sequential/dense_1/kernel'

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models

# ----------------------------
# 1️⃣ Dataset Yolu ve Yükleme
# ----------------------------
DATASET_DIR = "C:/Users/atunc/OneDrive/Belgeler/lung-cancer/dataset"

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

# ----------------------------
# 2️⃣ Sınıf Sayısını Alma
# ----------------------------
class_names = os.listdir(DATASET_DIR)
num_classes = len(class_names)
print("Sınıf sayısı:", num_classes)

# ----------------------------
# 3️⃣ Veri Ön İşleme
# ----------------------------
AUTOTUNE = tf.data.AUTOTUNE

# Normalize (0-1 aralığına)
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Performansı artırmak için cache ve prefetch
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ----------------------------
# 4️⃣ Model Oluşturma
# ----------------------------
model = models.Sequential([
    layers.Input(shape=(224,224,3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Sınıf sayısı
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ----------------------------
# 5️⃣ Modeli Eğitme
# ----------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# ----------------------------
# 6️⃣ Modeli Kaydetme
# ----------------------------
model.save("lung_cancer_model.h5")
print("Model kaydedildi: lung_cancer_model.h5")


Found 15000 files belonging to 3 classes.
Using 12000 files for training.
Found 15000 files belonging to 3 classes.
Using 3000 files for validation.
Sınıf sayısı: 3


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 137s 348ms/step - accuracy: 0.7666 - loss: 0.5952 - val_accuracy: 0.8853 - val_loss: 0.3007
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 130s 345ms/step - accuracy: 0.8985 - loss: 0.2497 - val_accuracy: 0.9217 - val_loss: 0.1937
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 122s 325ms/step - accuracy: 0.9229 - loss: 0.1875 - val_accuracy: 0.9290 - val_loss: 0.1892
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 119s 318ms/step - accuracy: 0.9383 - loss: 0.1574 - val_accuracy: 0.9400 - val_loss: 0.1584
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 116s 308ms/step - accuracy: 0.9612 - loss: 0.1083 - val_accuracy: 0.9457 - val_loss: 0.1594
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 116s 310ms/step - accuracy: 0.9715 - loss: 0.0787 - val_accuracy: 0.9427 - val_loss: 0.1789
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 115s 307ms/step - accuracy: 0.9796 - loss: 0.0603 - val_accuracy: 0.9380 - val_loss: 0.2142
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 120s 320ms/step - accuracy: 0.9887 -

Model kaydedildi: lung_cancer_model.h5


In [2]:
print(class_names)

['adenocarcinoma', 'benign', 'squamous_cell_carcinoma']
